In [1]:
# AWS utility functions for Resonate
## Video to audio convertor and store the video in firebase
# Author: Sartaj and Madhu

import boto3
import dotenv
import os
import time
import pandas as pd
import json
import webvtt
dotenv.load_dotenv()

def create_client(region_name: str = 'us-east-2') -> boto3.client:
    """
    Create and return an AWS Transcribe client with the specified or default AWS region.

    :param region_name: The AWS region where the Transcribe client will be created (default is 'us-east-2').
    :type region_name: str
    :return: An AWS Transcribe client and an S3 client.
    :rtype: boto3.clients
    """
    AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
    AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

    session = boto3.Session(
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
        region_name = region_name
        )
    return session.client('transcribe'), session.client('s3')



def create_s3_bucket(s3: boto3.client, bucket_name: str, region: str = 'us-east-2') -> bool:
    """
    Create an S3 bucket using the provided AWS S3 client if it doesn't exist.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param bucket_name: The name of the S3 bucket.
    :type bucket_name: str
    :param region: The AWS region where the S3 bucket should be created (default is 'us-east-1').
    :type region: str
    :return: True if the S3 bucket is successfully created or already exists, else False.
    :rtype: bool
    """
    try:
        # Attempt to create the bucket with a specific location constraint
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
        print(f"S3 bucket '{bucket_name}' created successfully.")
        return True
    except s3.exceptions.BucketAlreadyExists as e:
        print(f"S3 bucket '{bucket_name}' already exists.")
        return True
    except Exception as e:
        print(f"Error creating S3 bucket '{bucket_name}': {e}")
        return False


def upload_to_s3(s3: boto3.client, file_path: str, bucket_name: str, object_name=None) -> str:
    """
    Upload a file to an S3 bucket using the provided AWS S3 client, and create the bucket if it doesn't exist.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param file_path: The local path of the file to upload.
    :type file_path: str
    :param bucket_name: The name of the S3 bucket.
    :type bucket_name: str
    :param object_name: (Optional) The object name in the S3 bucket. If not specified, the file name will be used.
    :type object_name: str
    :return: The URI of the uploaded file (format: s3://bucket_name/object_name).
    :rtype: str
    """
    if object_name is None:
        object_name = file_path
        
    try:
        object_name = os.path.basename(file_path)
        s3.upload_file(file_path, bucket_name, object_name)
        uri = f"s3://{bucket_name}/{object_name}"
        print(f"File '{file_path}' uploaded successfully to '{uri}'")
        return uri
    except Exception as e:
        print(f"Error uploading file '{file_path}' to '{bucket_name}/{object_name}': {e}")
        return ""


def download_from_s3(s3: boto3.client, object_name: str, bucket_name: str = 'resonate-output', local_directory: str = '.') -> bool:
    """
    Download a file from an S3 bucket to a local directory.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param object_name: The object name in the S3 bucket.
    :type object_name: str
    :param bucket_name: The name of the S3 bucket.
    :type bucket_name: str
    :param local_directory: The local directory where the file should be saved (default is current directory).
    :type local_directory: str
    :return: True if the file was downloaded successfully, else False.
    :rtype: bool
    """
    local_file_json = f"{local_directory}/{object_name}.json"
    local_file_vtt = f"{local_directory}/{object_name}.vtt"

    try:
        # Download the file
        s3.download_file(bucket_name, object_name + '.json', local_file_json)
        s3.download_file(bucket_name, object_name + '.vtt', local_file_vtt)
        print(f"File '{object_name}' (JSON) downloaded successfully to '{local_file_json}'")
        print(f"File '{object_name}' (VTT) downloaded successfully to '{local_file_vtt}'")
        return True
    except Exception as e:
        print(f"Error downloading file '{object_name}' from '{bucket_name}': {e}")
        return False


def delete_from_s3(s3: boto3.client, bucket_name: str, object_name: str) -> bool:
    """
    Delete a file from an S3 bucket using the provided AWS S3 client.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param bucket_name: The name of the S3 bucket.
    :type bucket_name: str
    :param object_name: The object name in the S3 bucket.
    :type object_name: str
    :return: True if the file was deleted successfully, else False.
    :rtype: bool
    """
    try:
        s3.delete_object(Bucket=bucket_name, Key=object_name)
        print(f"File '{object_name}' deleted successfully from '{bucket_name}'")
        return True
    except Exception as e:
        print(f"Error deleting file '{object_name}' from '{bucket_name}': {e}")
        return False


import boto3

def delete_s3_bucket(s3, bucket_name):
    """
    Delete an S3 bucket along with its contents using the provided AWS S3 client.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param bucket_name: The name of the S3 bucket.
    :type bucket_name: str
    :return: True if the S3 bucket and its contents were deleted successfully, else False.
    :rtype: bool
    """    
    try:
        # List all objects in the bucket
        objects = s3.list_objects(Bucket=bucket_name).get('Contents', [])
        
        # Delete each object in the bucket
        for obj in objects:
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
            print(f"Object '{obj['Key']}' deleted successfully from '{bucket_name}'")

        # Delete the bucket
        s3.delete_bucket(Bucket=bucket_name)
        print(f"S3 bucket '{bucket_name}' and its contents deleted successfully.")
        return True
    except Exception as e:
        print(f"Error deleting S3 bucket '{bucket_name}': {e}")
        return False  
  


def transcribe_audio(transcribe_client: boto3.client, uri: str, output_bucket: str, transcribe_job_name: str='job')-> dict:
    """
    Start a transcription job for audio stored in an S3 bucket using the AWS Transcribe service.

    :param s3: The AWS S3 client used to interact with S3 services.
    :type s3: boto3.client
    :param URI: The URI of the audio file in the S3 bucket.
    :type URI: str
    :return: The response from the Transcribe service containing information about the transcription job.
    :rtype: dict
    """
    response = transcribe_client.start_transcription_job(
        TranscriptionJobName = transcribe_job_name,
        LanguageCode = 'en-US',
        MediaFormat = 'wav',
        Settings={
            'ShowSpeakerLabels': True,
            'MaxSpeakerLabels': 10,
            'ChannelIdentification': False,
        },  
        Media = {
            'MediaFileUri': uri
        },
        Subtitles={
            'Formats': ['vtt']
        },
        OutputBucketName=output_bucket,
    )
    return response

    

In [2]:
transcribe_client, s3_client = create_client()

In [3]:
print(create_s3_bucket(s3_client, 'resonate-input'))
print(create_s3_bucket(s3_client, 'resonate-output'))

Error creating S3 bucket 'resonate-input': An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
False
Error creating S3 bucket 'resonate-output': An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
False


In [7]:
URI = upload_to_s3(s3_client, 'C:/Users/madhu/Desktop/SeattleUniversity/capstone/resonate_madhu/Resonate/Notebooks/Finance & Corporate Committee - Zoom Meeting.wav/Finance_Corporate_Committee.wav', 'resonate-input')

File 'C:/Users/madhu/Desktop/SeattleUniversity/capstone/resonate_madhu/Resonate/Notebooks/Finance & Corporate Committee - Zoom Meeting.wav/Finance_Corporate_Committee.wav' uploaded successfully to 's3://resonate-input/Finance_Corporate_Committee.wav'


In [8]:
URI

's3://resonate-input/Finance_Corporate_Committee.wav'

In [9]:
input_bucket = 'resonate-input'
output_bucket = 'resonate-output'  
transcribe_job_name = 'Finance_Corporate_Committee'
file = 'Finance_Corporate_Committee.wav'

In [10]:
transcribe_audio(transcribe_client, URI, output_bucket, transcribe_job_name=transcribe_job_name)

{'TranscriptionJob': {'TranscriptionJobName': 'Finance_Corporate_Committee',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://resonate-input/Finance_Corporate_Committee.wav'},
  'StartTime': datetime.datetime(2024, 2, 10, 10, 15, 15, 770000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 2, 10, 10, 15, 15, 731000, tzinfo=tzlocal()),
  'Settings': {'ShowSpeakerLabels': True,
   'MaxSpeakerLabels': 10,
   'ChannelIdentification': False},
  'Subtitles': {'Formats': ['vtt']}},
 'ResponseMetadata': {'RequestId': '4dba41db-402f-4519-bbfb-45deca679b9d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4dba41db-402f-4519-bbfb-45deca679b9d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '418',
   'date': 'Sat, 10 Feb 2024 18:15:15 GMT'},
  'RetryAttempts': 0}}

In [61]:
transcribe_client.list_transcription_jobs(
    Status='COMPLETED',
    JobNameContains='string',
)

{'Status': 'COMPLETED',
 'TranscriptionJobSummaries': [],
 'ResponseMetadata': {'RequestId': '5cbd6a97-566d-45f5-8c81-3583347ebc82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5cbd6a97-566d-45f5-8c81-3583347ebc82',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '53',
   'date': 'Thu, 08 Feb 2024 19:51:14 GMT'},
  'RetryAttempts': 0}}

In [11]:
print(download_from_s3(s3_client, transcribe_job_name, output_bucket, local_directory='.'))

File 'Finance_Corporate_Committee' (JSON) downloaded successfully to './Finance_Corporate_Committee.json'
File 'Finance_Corporate_Committee' (VTT) downloaded successfully to './Finance_Corporate_Committee.vtt'
True


In [33]:
s3_client.close()

In [12]:
transcribe_client.delete_transcription_job(
    TranscriptionJobName=transcribe_job_name
)

{'ResponseMetadata': {'RequestId': '4623772e-9967-4990-b636-48d5f90e711a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4623772e-9967-4990-b636-48d5f90e711a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 10 Feb 2024 18:23:22 GMT'},
  'RetryAttempts': 0}}

In [13]:
print(delete_s3_bucket(s3_client, input_bucket))
print(delete_s3_bucket(s3_client, output_bucket))

Object 'ES2014c.Mix-Headset.wav' deleted successfully from 'resonate-input'
Object 'Finance_Corporate_Committee.wav' deleted successfully from 'resonate-input'
S3 bucket 'resonate-input' and its contents deleted successfully.
True
Object '.write_access_check_file.temp' deleted successfully from 'resonate-output'
Object 'Finance_Corporate_Committee.json' deleted successfully from 'resonate-output'
Object 'Finance_Corporate_Committee.vtt' deleted successfully from 'resonate-output'
S3 bucket 'resonate-output' and its contents deleted successfully.
True


In [14]:
file_name="Finance_Corporate_Committee"

In [15]:
with open(f'./{file_name}.json', 'r') as f: ## change the file path to the json file saved path that you got from AWS
        data = json.load(f)

    # Extract the relevant information from the JSON file
segments = data['results']['speaker_labels']['segments']
rows = []
for segment in segments:
        start_time = float(segment['start_time'])/60
        end_time = float(segment['end_time'])/60
        speaker_label = segment['speaker_label']
        rows.append([start_time, end_time, speaker_label])
df = pd.DataFrame(rows, columns=['start_time', 'end_time', 'speaker_label'])
    
    # Load the webvtt file
subtitles = webvtt.read(f'./{file_name}.vtt') ## change the file path to the .vtt file saved path that you got from AWS

    # Initialize an empty list to store the captions
data = []

    # Loop through the captions and extract the information

for subtitle in subtitles:
        start_time = subtitle.start.split(':')
        end_time = subtitle.end.split(':')

    # Convert start and end time to minutes
        start_minutes = int(start_time[0])*60 + int(start_time[1]) + float(start_time[2])/60
        end_minutes = int(end_time[0])*60 + int(end_time[1]) + float(end_time[2])/60

        text = subtitle.text.strip()

    # Append the information to the data list
        data.append((start_minutes, end_minutes, text))


# Create a pandas dataframe from the data list
titles = pd.DataFrame(data, columns=['start_time', 'end_time', 'text'])
    # Merge the two tables based on start_time
merged = pd.merge_asof(titles.sort_values('start_time'), df.sort_values('start_time'), on='start_time', direction='backward')

# Drop rows with NaN values in the speaker_label column
merged = merged.dropna(subset=['speaker_label'])

# Rename the columns
merged = merged[['start_time', 'end_time_x', 'speaker_label', 'text']]  
merged.columns = ['start_time', 'end_time', 'speaker_label', 'text']

# Reset the index
merged = merged.reset_index(drop=True)
merged.to_csv(f'./{file_name}.csv')

#### Video to audio convertor

In [6]:
import moviepy.editor as mp
def video_to_audio(in_path, out_path):
    """Convert video file to audio file"""
    
    video = mp.VideoFileClip(in_path)
    video.audio.write_audiofile(out_path)
# Video to audio
video_to_audio('C:/Users/madhu/Desktop/SeattleUniversity/capstone/resonate_madhu/Resonate/Notebooks/architecture/2023-10-17_New_diffs_architecture_blueprint.mp4', '2023-10-17_New_diffs_architecture_blueprint.wav')



MoviePy - Writing audio in 2023-10-17_New_diffs_architecture_blueprint.wav


MoviePy - Done.


#### connect to firebase

In [12]:
json_data= {
    "index": "langchain-retrieval-transcript",
    "namespace": "new_namespace_1",
    "last_meeting_no": 1,
    "last_conversation_no": 145,
    "unique_meeting_members": [
        "test2",
        "test1"
    ],
    "meetings": [
        {
            "meeting_no": 1,
            "meeting_date": "2024-01-24 13:23:38",
            "meeting_video_file": True,
            "meeting_members": [
                "test1",
                "test2"
            ]
            
        },
       
    ]
}



In [13]:
import firebase_admin
from firebase_admin import credentials, db, storage
def initialize_firebase():
    try:
        cred = credentials.Certificate("resonate.private_key.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://your-project-id.firebaseio.com',
            'storageBucket': 'resonate-917af.appspot.com'
        })
        print("Firebase initialization successful")
    except Exception as e:
        print(f"Firebase initialization failed: {e}")

In [29]:
from firebase_admin import credentials, initialize_app,db storage

def initialize_firebase():
    try:
        cred = credentials.Certificate("resonate.private_key.json")
        initialize_app(cred, {
            'databaseURL': 'https://resonate-917af-default-rtdb.firebaseio.com/',
            'storageBucket': 'resonate-917af.appspot.com'})
        print("Firebase initialization successful")
    except Exception as e:
        print(f"Firebase initialization failed: {e}")
def push_json_to_database(json_data, database_path='/'):
    try:
        ref = db.reference(database_path)
        ref.set(json_data)
        print(f"JSON data pushed to '{database_path}' in Realtime Database")
    except Exception as e:
        print(f"Error pushing JSON data to database: {e}")


def upload_video_to_storage(local_path, storage_path):
    try:
        bucket = storage.bucket()
        blob = bucket.blob(storage_path)
        blob.upload_from_filename(local_path)

        # Opt: if you want to make public access from the URL
        blob.make_public()

        print(f"Video file uploaded to '{storage_path}' in Cloud Storage")
        print("Your file URL:", blob.public_url)
        return blob.public_url
    except Exception as e:
        print(f"Error uploading video file to Cloud Storage: {e}")
        return None

# Example usage:
if __name__ == "__main__":
    try:
        initialize_firebase()

        json_data= {
    "index": "langchain-retrieval-transcript",
    "namespace": "new_namespace_1",
    "last_meeting_no": 1,
    "last_conversation_no": 1,
    "unique_meeting_members": [
        "test2",
        "test1"
    ],
    "meetings": [
        {
            "meeting_no": 1,
            "meeting_date": "2024-01-24 13:23:38",
            "meeting_video_file": True,
            "meeting_members": [
                "test1",
                "test2"
            ]
            
        },
       
    ]
}


        database_path = '/meeting_metadata'
        push_json_to_database(json_data, database_path)

        video_local_path = 'Product Marketing Meeting (weekly) 2021-06-28.mp4'
        video_storage_path = 'Product Marketing Meeting (weekly) 2021-06-28.mp4'
        uploaded_url = upload_video_to_storage(video_local_path, video_storage_path)

        if uploaded_url:
            print(f"Video file uploaded successfully. Public URL: {uploaded_url}")

    except KeyboardInterrupt:
        print("\nScript execution interrupted.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        firebase_admin.delete_app(firebase_admin.get_app())  # Clean up Firebase app


Firebase initialization successful
JSON data pushed to '/meeting_metadata' in Realtime Database
Video file uploaded to 'Product Marketing Meeting (weekly) 2021-06-28.mp4' in Cloud Storage
Your file URL: https://storage.googleapis.com/resonate-917af.appspot.com/Product%20Marketing%20Meeting%20%28weekly%29%202021-06-28.mp4
Video file uploaded successfully. Public URL: https://storage.googleapis.com/resonate-917af.appspot.com/Product%20Marketing%20Meeting%20%28weekly%29%202021-06-28.mp4


In [21]:
video_file= 'Product Marketing Meeting (weekly) 2021-06-28.mp4'
push_json_to_database(json_data, '/')

upload_video_to_storage(video_file, f"{video_file}")



JSON data pushed to '/' in Realtime Database
Error uploading video file to Cloud Storage: Storage bucket name not specified. Specify the bucket name via the "storageBucket" option when initializing the App, or specify the bucket name explicitly when calling the storage.bucket() function.
